In [375]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage import filters
import scipy.ndimage as sci_image
from skimage import measure as meas
import matplotlib.pyplot as plt
from scipy import signal as sci_signal

### Templates for Identification of Characters

In [376]:
def identify_characters(full_path):
    
    input_images = {}
    
    begin = ord("A")
    end = ord("Z") + 1
    for i in range(begin,end):
        input_images[chr(i)] = np.array(Image.open(full_path + "\\" + chr(i) + ".bmp"))
    
    begin = ord("0")
    end = ord("9") + 1
    for i in range(begin,end):
        input_images[chr(i)] = np.array(Image.open(full_path + "\\" +chr(i) + ".bmp"))
    
    characters = np.array(list(input_images.values()))
    return characters

### Letter Detection

In [377]:
def character_listings(ind):
    if ind == 1 or ind == 2:
        letter = 'A'
    elif ind == 3 or ind == 4:
        letter = 'B'
    elif ind == 5:
        letter = 'C'
    elif ind == 6 or ind == 7:
        letter = 'D'
    elif ind == 8:
        letter = 'E'
    elif ind == 9:
        letter = 'F'
    elif ind == 10:
        letter = 'G'
    elif ind == 11:
        letter = 'H'
    elif ind == 12:
        letter = 'I'
    elif ind == 13:
        letter = 'J'
    elif ind == 14:
        letter = 'K'
    elif ind == 15:
        letter = 'L'
    elif ind == 16:
        letter = 'M'
    elif ind == 17:
        letter = 'N'
    elif ind == 18 or ind == 19:
        letter = 'O'
    elif ind == 20 or ind == 21:
        letter = 'P'
    elif ind == 22 or ind == 23:
        letter = 'Q'
    elif ind == 24 or ind == 25:
        letter = 'R'
    elif ind == 26:
        letter = 'S'
    elif ind == 27:
        letter = 'T'
    elif ind == 28:
        letter = 'U'
    elif ind == 29:
        letter = 'V'
    elif ind == 30:
        letter = 'W'
    elif ind == 31:
        letter = 'X'
    elif ind == 32:
        letter = 'Y'
    elif ind == 33:
        letter = 'Z'
    elif ind == 34:
        letter = '1'
    elif ind == 35:
        letter = '2'
    elif ind == 36:
        letter = '3'
    elif ind == 37 or ind == 38:
        letter = '4'
    elif ind == 39:
        letter = '5'
    elif ind == 40 or ind == 41 or ind == 42:
        letter = '6'
    elif ind == 43:
        letter = '7'
    elif ind == 44 or ind == 45:
        letter = '8'
    elif ind == 46 or ind == 47 or ind == 48:
        letter = '9'
    else:
        letter = '0'

    return letter


In [378]:
def detect_letter(image_original):
    image_resized = image_original.resize([42,24])
    image_resized_array = np.array(image_resized)

    image_templates = identify_characters("C:\\Users\\amith\\Documents\\GitHub\\mvsr-data-scientist\\Detection of Vehicle Number on Licence Plate\\data")

    corr_values = []
    for i in image_templates:
        corr_matrix = sci_signal.correlate2d(in1=i,in2=image_resized_array).max()
        corr_values.append(corr_matrix)

    max_corr = max(corr_values)
    pos = np.where(corr_values == max_corr)

    charac = character_listings(pos)

    return charac,pos

### Detection of Number Plate

In [379]:
# Path where our image is located
path = "C:\\Users\\amith\\Documents\\Number Plate Recognition\\Number Plate Images\\image2.png"

# Opening of Image
image = Image.open(path)

# Conversion of Image to grayscale
image_grayscale = image.convert("L")

# Making the pixel values binary (either 0's or 1's) in the image
image_binarized = (np.array(image_grayscale) / 255.0).round()

# Applying Sobel Edge Detector to detect edges
image_edges_X = sci_image.sobel(image_binarized,axis=0)
image_edges_y = sci_image.sobel(image_binarized,axis=1)
image_edges = np.int64(np.sqrt(image_edges_X**2 + image_edges_y**2).round())

In [380]:
image_region_properties = meas.regionprops(label_image=image_edges)

label = []
bounding_box = []
area = []

property_count = 0

for i in image_region_properties:
    label.append(i.label)
    bounding_box.append(i.bbox)
    area.append(i.area)
    property_count += 1

max_area = max(area)
max_area_bounding_box = bounding_box[area.index(max_area)]
for i in range(property_count):
    if max_area < area[i]:
        max_area = area[i]
        max_area_bounding_box = bounding_box[i]

In [381]:
row_1, col_1, row_2, col_2 = max_area_bounding_box

image_cropped = image_binarized[row_1:row_2, col_1:col_2]

image_cropped_inverted = 1 - image_cropped
image_labelled, number_of_labels = sci_image.label(image_cropped_inverted)
component_sizes = sci_image.sum(image_cropped_inverted,image_labelled,range(number_of_labels+1))
mask = component_sizes > 500
image_filtered_before_inversion = np.int64(mask[image_labelled])
image_filtered = 1 - image_filtered_before_inversion

height_ref,width_ref = image_filtered.shape

In [382]:
image_region_properties_2 = meas.regionprops(label_image=image_filtered)

label_2 = []
bounding_box_2 = []
area_2 = []

property_count_2 = 0

for i in image_region_properties_2:
    label_2.append(i.label)
    bounding_box_2.append(i.bbox)
    area_2.append(i.area)
    property_count_2 += 1

number_on_plate = []
kkl = []

for i in range(property_count_2):
    bounding_box_2_height = image_region_properties_2[i].image.shape[0]
    bounding_box_2_width = image_region_properties_2[i].image.shape[1]

    if (bounding_box_2_width < (height_ref/2)) and (bounding_box_2_height > (height_ref/3)):
        letter,kk = detect_letter(image_region_properties_2[i].image)
        number_on_plate.append(letter)
        kkl.append(kk)

In [383]:
number_on_plate

[]

In [384]:
kkl

[]